In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
betting_moneyline_df = pd.read_csv("nba_betting_money_line.csv")
betting_moneyline_df.head()


,game_id,book_name,book_id,team_id,a_team_id,price1,price2
0,41100314,Pinnacle Sports,238,1610612759,1610612760,165.0,-183.0
1,41100314,5Dimes,19,1610612759,1610612760,165.0,-175.0
2,41100314,Bookmaker,93,1610612759,1610612760,160.0,-190.0
3,41100314,BetOnline,1096,1610612759,1610612760,165.0,-190.0
4,41100314,Bovada,999996,1610612759,1610612760,155.0,-175.0


In [3]:
games_all_df = pd.read_csv("nba_games_all.csv")
games_all_df.head()

,game_id,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,a_team_id,season_year,season_type,season
0,20800741,2009-02-06,SAC vs. UTA,1610612762,f,W,29.0,22.0,0.569,240,39.0,74.0,0.527,4.0,13.0,0.308,29.0,37.0,1.0,8.0,36.0,44,19.0,5.0,4.0,18.0,26.0,111,1610612758,2008,Regular Season,2008-09
1,20800701,2009-01-31,POR vs. UTA,1610612762,f,L,26.0,22.0,0.542,240,37.0,68.0,0.544,5.0,20.0,0.250,29.0,37.0,1.0,7.0,20.0,27,17.0,6.0,0.0,15.0,22.0,108,1610612757,2008,Regular Season,2008-09
2,20800584,2009-01-16,MEM vs. UTA,1610612762,f,W,24.0,16.0,0.600,240,37.0,71.0,0.521,7.0,19.0,0.368,20.0,27.0,1.0,7.0,26.0,33,23.0,9.0,3.0,15.0,22.0,101,1610612763,2008,Regular Season,2008-09
3,20800558,2009-01-12,IND @ UTA,1610612762,t,W,23.0,15.0,0.605,240,39.0,85.0,0.459,8.0,18.0,0.444,34.0,37.0,1.0,9.0,31.0,40,24.0,10.0,6.0,8.0,20.0,120,1610612754,2008,Regular Season,2008-09
4,20800440,2008-12-27,HOU vs. UTA,1610612762,f,L,18.0,14.0,0.563,290,49.0,100.0,0.490,5.0,13.0,0.385,12.0,18.0,1.0,11.0,29.0,40,35.0,13.0,7.0,9.0,27.0,115,1610612745,2008,Regular Season,2008-09


In [4]:
betting_spread_df = pd.read_csv("nba_betting_spread.csv")
betting_spread_df.head()

,game_id,book_name,book_id,team_id,a_team_id,spread1,spread2,price1,price2
0,21000358,Pinnacle Sports,238,1610612749,1610612742,7.5,-7.5,-106.0,-104.0
1,21000358,5Dimes,19,1610612749,1610612742,7.5,-7.5,-110.0,-110.0
2,21000358,Bookmaker,93,1610612749,1610612742,7.5,-7.5,-110.0,-110.0
3,21000358,BetOnline,1096,1610612749,1610612742,7.5,-7.5,-110.0,-110.0
4,21000358,Bovada,999996,1610612749,1610612742,8.0,-8.0,-115.0,-105.0


In [6]:
teams_df = pd.read_csv("nba_teams_all.csv")
teams_df.head()

,league_id,team_id,min_year,max_year,abbreviation
0,0,1610612737,1949.0,2018.0,ATL
1,0,1610612738,1946.0,2018.0,BOS
2,0,1610612739,1970.0,2018.0,CLE
3,0,1610612740,2002.0,2018.0,NOP
4,0,1610612741,1966.0,2018.0,CHI


In [13]:
def get_home_team(row):
  matchup = row['matchup']
  if '@' in matchup:
    at_index = matchup.index('@')
    return matchup[at_index+1:]
  else:
    vs_index = matchup.index('vs.')
    return matchup[:vs_index]

def get_away_team(row):
  matchup = row['matchup']
  if '@' in matchup:
    at_index = matchup.index('@')
    return matchup[:at_index]
  else:
    vs_index = matchup.index('vs.')
    return matchup[vs_index+4:]

def get_winner(row):
  if row['is_home'] == 'f':
    if row['wl'] == 'W':
      return row['away_team']
    else:
      return row['home_team']
  else:
    if row['wl'] == 'W':
      return row['home_team']
    else:
      return row['away_team']

games_all_df['home_team'] = games_all_df.apply(lambda row: get_home_team(row), axis=1)
games_all_df['away_team'] = games_all_df.apply(lambda row: get_away_team(row), axis=1)
games_all_df['winner'] = games_all_df.apply(lambda row: get_winner(row), axis=1)


games_all_df.head()

,game_id,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,a_team_id,season_year,season_type,season,home_team,away_team,winner
0,20800741,2009-02-06,SAC vs. UTA,1610612762,f,W,29.0,22.0,0.569,240,39.0,74.0,0.527,4.0,13.0,0.308,29.0,37.0,1.0,8.0,36.0,44,19.0,5.0,4.0,18.0,26.0,111,1610612758,2008,Regular Season,2008-09,SAC,UTA,UTA
1,20800701,2009-01-31,POR vs. UTA,1610612762,f,L,26.0,22.0,0.542,240,37.0,68.0,0.544,5.0,20.0,0.250,29.0,37.0,1.0,7.0,20.0,27,17.0,6.0,0.0,15.0,22.0,108,1610612757,2008,Regular Season,2008-09,POR,UTA,POR
2,20800584,2009-01-16,MEM vs. UTA,1610612762,f,W,24.0,16.0,0.600,240,37.0,71.0,0.521,7.0,19.0,0.368,20.0,27.0,1.0,7.0,26.0,33,23.0,9.0,3.0,15.0,22.0,101,1610612763,2008,Regular Season,2008-09,MEM,UTA,UTA
3,20800558,2009-01-12,IND @ UTA,1610612762,t,W,23.0,15.0,0.605,240,39.0,85.0,0.459,8.0,18.0,0.444,34.0,37.0,1.0,9.0,31.0,40,24.0,10.0,6.0,8.0,20.0,120,1610612754,2008,Regular Season,2008-09,UTA,IND,UTA
4,20800440,2008-12-27,HOU vs. UTA,1610612762,f,L,18.0,14.0,0.563,290,49.0,100.0,0.490,5.0,13.0,0.385,12.0,18.0,1.0,11.0,29.0,40,35.0,13.0,7.0,9.0,27.0,115,1610612745,2008,Regular Season,2008-09,HOU,UTA,HOU


In [16]:
moneyline_games_merge = pd.merge(left=betting_moneyline_df, right=games_all_df, left_on='game_id', right_on='game_id')
moneyline_games_merge.drop(columns=['game_id', 'book_id', 'team_id_x', 'a_team_id_x', 'matchup', 'team_id_y', 'wl', 'w', 'l', 'w_pct', 'min', 'a_team_id_y'])
moneyline_games_merge.head()

,game_id,book_name,book_id,team_id_x,a_team_id_x,price1,price2,game_date,matchup,team_id_y,is_home,wl,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,a_team_id_y,season_year,season_type,season,home_team,away_team,winner
0,41100314,Pinnacle Sports,238,1610612759,1610612760,165.0,-183.0,2012-06-02,SAS @ OKC,1610612760,t,W,10.0,3.0,0.769,240,44.0,78.0,0.564,5.0,13.0,0.385,16.0,21.0,1.0,7.0,34.0,41,27.0,5.0,5.0,12.0,15.0,109,1610612759,2011,Playoffs,2011-12,OKC,SAS,OKC
1,41100314,Pinnacle Sports,238,1610612759,1610612760,165.0,-183.0,2012-06-02,OKC vs. SAS,1610612759,f,L,10.0,2.0,0.833,240,41.0,82.0,0.500,11.0,23.0,0.478,10.0,16.0,1.0,7.0,24.0,31,17.0,5.0,2.0,10.0,20.0,103,1610612760,2011,Playoffs,2011-12,OKC,SAS,OKC
2,41100314,5Dimes,19,1610612759,1610612760,165.0,-175.0,2012-06-02,SAS @ OKC,1610612760,t,W,10.0,3.0,0.769,240,44.0,78.0,0.564,5.0,13.0,0.385,16.0,21.0,1.0,7.0,34.0,41,27.0,5.0,5.0,12.0,15.0,109,1610612759,2011,Playoffs,2011-12,OKC,SAS,OKC
3,41100314,5Dimes,19,1610612759,1610612760,165.0,-175.0,2012-06-02,OKC vs. SAS,1610612759,f,L,10.0,2.0,0.833,240,41.0,82.0,0.500,11.0,23.0,0.478,10.0,16.0,1.0,7.0,24.0,31,17.0,5.0,2.0,10.0,20.0,103,1610612760,2011,Playoffs,2011-12,OKC,SAS,OKC
4,41100314,Bookmaker,93,1610612759,1610612760,160.0,-190.0,2012-06-02,SAS @ OKC,1610612760,t,W,10.0,3.0,0.769,240,44.0,78.0,0.564,5.0,13.0,0.385,16.0,21.0,1.0,7.0,34.0,41,27.0,5.0,5.0,12.0,15.0,109,1610612759,2011,Playoffs,2011-12,OKC,SAS,OKC


In [ ]:
high_bets = moneyline_games_merge[moneyline_games_merge['price1'] >= 1000]
high_bets.value_counts('book_name')

#sns.scatterplot(data=high_bets, x='price1', y='price2', hue='book_name')

book_name
5Dimes             1272
Pinnacle Sports    1212
Bookmaker          1006
JustBet             876
Heritage            622
Sportsbetting       458
BetOnline           458
YouWager            340
Intertops           332
Bovada               56
dtype: int64